# First Class Functions
## Docstrings and Annotations
- docstring은 help()실행시 프린트된다.
- docstring은 `__doc__` 호출시 프린트된다.
- 별거 아니지만 소소한 지식: annotation string 형태를 줘도 작동한다.
- 참고로 annotation과 type hint는 완전히 같은 뜻은 아니다.

In [4]:
def my_func(a: "zzzzz"):
    pass

help(my_func)
print(my_func.__annotations__)

Help on function my_func in module __main__:

my_func(a: 'zzzzz')

{'a': 'zzzzz'}


## Lambda Expressions
- 람다는 anonymous functions 이다.
- 람다 문법: `lambda [parameter list]: expression`
- lambda는 closure가 아니다.

<img width="800" alt="스크린샷 2022-02-12 오후 4 24 41" src="https://user-images.githubusercontent.com/60768642/153701622-2108c6c2-51d5-4bb8-a80e-ee568194cc80.png">

```
lambda x: x ** 2
lambda x, y: x + y
lambda: "hello"
lambda s: s[::-1].upper()
print(type(lambda x: x ** 2))
```

- lambda 함수 예시.

In [13]:
my_func = lambda x: x ** 2
print(my_func(2))
print(my_func(3))

4
9


위 함수는 아래 함수와 같다.

```
def my_func(x):
    return x ** 2
```

- 요런 것도 가능하다.

In [14]:
def apply_func(x, fn):
    return fn(x)

print(apply_func(2, lambda x: x + 2))

4


### Lambda Limitations
#### No assignments
- `lambda x: x=5`는 불가능하다
#### No annotations
- 주석 불가능
#### Single logical line of code
- line break(\\사용)는 가능하지만 논리적으로 하나의 라인이어야 한다.

## Lambda and Sorting
- sorted함수에는 key에 함수를 넘겨 sort 알고리즘을 정할 수 있다.

In [19]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



In [21]:
# default sort
l = ["c", "D", "Z", "z", "a", "A"]
sorted(l)

['A', 'D', 'Z', 'a', 'c', 'z']

In [22]:
# case insensitive sorting
l = ["c", "D", "Z", "z", "a", "A"]
sorted(l, key=lambda x: x.upper())

['a', 'A', 'c', 'D', 'Z', 'z']

In [25]:
# dict의 value기준 정렬
my_dict = {"banana": 3, "apple": 5, "pear": 1}
sorted(my_dict)

['apple', 'banana', 'pear']

In [26]:
my_dict = {"banana": 3, "apple": 5, "pear": 1}
sorted(my_dict, key=lambda v: my_dict[v])

['pear', 'banana', 'apple']

In [66]:
import random

l = [i for i in range(1, 11)]
sorted(l, key=lambda x: random.random())

[2, 10, 1, 5, 8, 9, 3, 4, 6, 7]

## Function Introspection
- `__doc__`, `__annotations__`
- 파이썬에서 함수는 first-class object이기 때문에 아래 같은 것도 가능하다.(attribute 소유 가능)

In [13]:
def my_func(a, b):
    pass

my_func.name = "this is my func!"
my_func.age = 15

print(my_func.name)
print(my_func.age)
"name" in dir(my_func) and "age" in dir(my_func)

this is my func!
15


True

#### `__name__`, `__defaults__`, `kwdefaults__`
- `__name__`: name of function
- `__defaults__`: tuple containing positional parameter defaults
- `kwdefaults__`: dictionary containing keyward-only parameter defaults

In [16]:
def my_func(a, b=2, c=3, *, kw1, kw2=10):
    pass

print(my_func.__name__)
print(my_func.__defaults__)
print(my_func.__kwdefaults__)

my_func
(2, 3)
{'kw2': 10}


#### `__code__`
- `__code__` object itself has various properties, which include:
    - co_varnames: param and local variables. (param first, local variable after)
    - co_argcount: number of param. (doesn't count `*args`, `**kwargs`)

In [18]:
def my_func(a, b=2, *args, **kwargs):
    i = 111
    b = min(i, b)
    return a * b

print(my_func.__code__.co_varnames)
print(my_func.__code__.co_argcount)

('a', 'b', 'args', 'kwargs', 'i')
2
b'd\x01}\x04t\x00|\x04|\x01\x83\x02}\x01|\x00|\x01\x14\x00S\x00'


#### inspect module(import inspect)
- ismethod(obj): 클래스에 종속된 메서드일 경우 True
- isfunction(obj): (클래스에 종속되지 않은) 함수일 경우(=callable인경우) True
- isroutine(obj): 함수, 메서드인 경우 True
- 많은 IDE가 TODO comment를 색칠해서 보여주곤 하는데, 이 때 inspect.getcomments를 사용한다.

In [28]:
def my_func(a, b=2, *args, **kwargs):
    """this is docs"""
    i = 111  # some coment
    # TODO: todo
    b = min(i, b)
    # comment!!!
    return a * b

import inspect
print(inspect.getsource(my_func))
print(inspect.getmodule(my_func))
print(inspect.getmodule(print))
print(inspect.getcomments(my_func))
# 근데 getcomments는 왜 값을 못읽고 None이 뜨는거지??

def my_func(a, b=2, *args, **kwargs):
    """this is docs"""
    i = 111  # some coment
    # TODO: todo
    b = min(i, b)
    # comment!!!
    return a * b

<module '__main__'>
<module 'builtins' (built-in)>
None


#### Callable Signatures
- inspect.signature(my_func) -> Signature instance
- contains and attribute called parameters
- Essentially a dictionary of parameter names(keys), and metadata about the parameters(vaules)
    - key -> parameter name
    - values -> object with attributes such as name, default, annotation, kind
- kind
    - POSITIONAL_OR_KEYWORD
    - VAR_POSITIONAL
    - KEYWORD_ONLY
    - VAR_KEYWORD
    - POSITIONAL_ONLY

In [33]:
def my_func(
        a: str,
        b: int,
        c=3,
        *args,
        **kwargs):
    """docs"""
    i = 111
    b = min(i, b)
    return a * b

import inspect

print(inspect.signature(my_func))

for param in inspect.signature(my_func).parameters.values():
    print("name", param.name)
    print("default", param.default)
    print("annotation", param.annotation)
    print("kind", param.kind)









(a: str, b: int, c=3, *args, **kwargs)
name a
default <class 'inspect._empty'>
annotation <class 'str'>
kind POSITIONAL_OR_KEYWORD
name b
default <class 'inspect._empty'>
annotation <class 'int'>
kind POSITIONAL_OR_KEYWORD
name c
default 3
annotation <class 'inspect._empty'>
kind POSITIONAL_OR_KEYWORD
name args
default <class 'inspect._empty'>
annotation <class 'inspect._empty'>
kind VAR_POSITIONAL
name kwargs
default <class 'inspect._empty'>
annotation <class 'inspect._empty'>
kind VAR_KEYWORD


## Callables
- any object that can be called using the () operator
#### different types of callables
- built-in functions: print, len 등등
- built-in methods: str.upper, list.append 등등
- user-defined functions: def, labmda 등등
- methods
- classes:
  - instance 생성시 MyClass() 형태로 호출 -> callable임
  - 참고로 클래스 호출시 `__new__` ->`__init__`형태로 호출된다.
  - `__new__`에서 이미 인스턴스가 생성된다. `__init__`의 첫 param self에 new에서 생성된 instance가 전달됨.
- class instances: `__call__`을 정의하면 인스턴스도 callable이된다.



## Map, Filter, Zip and List Comprehensions
### higher order functions
- a function that takes a function as a parameter and/or returns a function as its return value
- map, filter는 higher order function인데 list comprehension과 generator expressions으로 대체 가능하다(modern alternative)

### map(func, *iterables)
- *iterables: a variable number of iterable objects
- func: some function that takes as many arguments as there are iterable objects passed to iterables
- map function return an iterator that caculates the function applied to each element of the iterables
- The iterator stops as soon as one of the iterables has benn exhausted so, unequal length iterables can be used

In [6]:
l = [1,2,3]

def sq(x):
    return x ** 2

print(map(sq, l))
print(list(map(sq, l)))

l1 = [1, 2, 3]
l2 = [10, 20, 30]
l3 = [100, 200, 300]

def add(x, y):
    return x + y

print(list(map(add, l1, l2)))

res = map(lambda x, y, z: x + y + z, l1, l2, l3)
print(list(res))

[1, 4, 9]
[11, 22, 33]
[111, 222, 333]


### filter(func, iterable)
- iterable: a single iterable
- func: some function that takes a single argument
- filter() will then return an iterator that contains all the elements of the iterable for which the function called on it is Truthy
- If the function is None, it simply returns the elements of iterable that are truthy


In [13]:
l = [i for i in range(5)]
res = filter(None, l)
print(list(res))

def is_even(number):
    return number % 2 == 0

res = filter(is_even, l)
print(list(res))

res = filter(lambda x: x>10, [1,5,10,11])
print(list(res))


[1, 2, 3, 4]
[0, 2, 4]
[11]


### zip

In [17]:
l1 = [1,2,3]
l2 = [4,5,6]
l3 = [6,7,8,9,10]

res = zip(l1, l2)
print(list(res))

res = zip(l1, l2, l3)
print(list(res))

res = zip(l1, l3)
print(list(res))

[(1, 4), (2, 5), (3, 6)]
[(1, 4, 6), (2, 5, 7), (3, 6, 8)]
[(1, 6), (2, 7), (3, 8)]


### combining map, filter

In [20]:
l = range(10)
res = filter(lambda y: y < 25, map(lambda x: x**2, l))
print(list(res))

# list comprehension으로 대체하기(더 가독성이 좋음)
res = [x**2 for x in l if x **2 < 25]
print(res)


[0, 1, 4, 9, 16]
[0, 1, 4, 9, 16]


## Reducing Functions

## Partial Functions
- idea behind partial functions: how to reduce number of required args
- my_func의 args를 줄이기 위해 아래처럼 할 수 있다.

In [2]:
def my_func(a,b,c):
    print(a,b,c)


def fn(b, c):
    return my_func(10, b, c)

f = lambda b, c: my_func(10, b, c)

from functools import partial

f = partial(my_func, 10)

f(2,3)

10 2 3


#### partial 사용 예제

In [6]:
def my_func(a, b, *args, k1, k2, **kwargs):
    print(a,b,args,k1,k2,kwargs)

f = partial(my_func, 10, k1="a")


def pow(base, exponent):
    return base ** exponent

square = partial(pow, exponent=2)
cube = partial(pow, exponent=3)

print(square(5))
print(cube(5))
print(cube(base=5))
print(square(5, exponent=3))

25
125
125
125


#### 주의사항
<img width="1037" alt="스크린샷 2022-02-16 오전 9 16 51" src="https://user-images.githubusercontent.com/60768642/154171957-0281d0ed-45f1-4a2f-93be-2631f51264af.png">


## The operator modules